In [ ]:
import os
os.chdir('..')

In [ ]:
import model_config 
from model_packages import *
from model_utils import *

import add_counter_location_info 
# import add_census_features 
import add_census_features_2021
import add_green_blue_infrastructure
import add_land_classification
import add_land_habitat_data
import add_dog_occupancy
import add_weather
import add_POIs
import add_strava_data
import add_counter_data


In [ ]:
# Running each main function to generate static features data set
add_counter_location_info.main()
# add_census_features.main()
add_census_features_2021.main()
add_green_blue_infrastructure.main()
add_land_classification.main()
add_land_habitat_data.main()
add_dog_occupancy.main()
add_POIs.main()

# access config file to loop through providers and datasets
config= add_counter_location_info.get_config_file_paths('./scripts/config.json')

# process people_counter data for each provider
for provider, info in config.items():
    print(f'>>>>>>>>>>>>>{info["pc_path"]}')
    info['cut_off_year']
    add_counter_data.main(provider, info['pc_path'].strip("'"), info['cut_off_year'])

# process strava data for each provider
for provider, info in config.items():
    print(f'>>>>>>>>>>>>>{info["strava_path"]}')
    add_strava_data.main(provider, info['strava_path'])
    


# combine counts data for all providers
dfs=[]
for provider, info in config.items():
    df= pd.read_pickle(data_folder+f'pc_and_strava_{provider}.pkl')
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['Date']=combined_df['Date'].astype(str)


# merge with weather features
add_weather.main()
weather= pd.read_pickle(data_folder+'weather_data.pkl')
weather['Date']=weather['Date'].astype(str)
dynamic= weather.merge(combined_df, on= ['Date', 'site'], how = 'right')

# merge with static features
static= pd.read_pickle(data_folder+'static_data.pkl')
static_and_dynamic_features= dynamic.merge(static, left_on= 'site', right_on='counter', how= 'inner')

# save data of static and dynamic features for modelling
static_and_dynamic_features.to_pickle(data_folder+f'static_and_dynamic_features_{bufr_zones_mrts}.pkl')

# Plotting for number of available counters

In [ ]:
df_raw, df_monthly_ne, df_smooth, up, low= add_counter_data.main('ne',"./data/counter_data/natural_england_national_trails_people_counters_data_2020-01_to_2023-09.csv", 2020)

In [ ]:
df_raw, df_monthly_ndw, df_smooth, up, low= add_counter_data.main('ndw',"./data/counter_data/north_downs_way_people_counters_data_2020-01_to_2023-09.csv", 2021)

In [ ]:
df_raw, df_monthly_t, df_smooth, up, low= add_counter_data.main('t',"./data/counter_data/thames_reformatted.csv", 2020)

In [ ]:
df_raw, df_monthly_d, df_smooth, up, low= add_counter_data.main('d',"./data/counter_data/Dorset_refromatted.csv", 2020)

In [ ]:
df2= df_monthly_ne.merge(df_monthly_ndw, on='Date', how= 'outer')
df2= df2.merge(df_monthly_t, on='Date',how= 'outer')
df2= df2.merge(df_monthly_d, on='Date', how= 'outer')
df2

In [ ]:
df_m2= df2.reset_index().melt(id_vars=['Date'], var_name='site', value_name='people_count')

In [ ]:
df_m2.dropna().groupby('Date')['site'].count().plot(style='-o')
plt.ylabel('Number of monitoring sites with training data available')